In [7]:
from llm.LLMModel import *
from ner.llm_ner.prompt_techniques.pt_abstract import PromptTechnique
from ner.llm_ner.prompt_techniques.pt_discussion import PT_OutputList
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_wrapper import PT_Wrapper
from ner.llm_ner.few_shots_techniques import *
%reload_ext autoreload
%autoreload 2
from ner.Datasets.Conll2003Dataset import Conll2003Dataset, load_conll_dataset
from ner.evaluating_confidence import get_logits_for_tags, values_to_find


# from llm.llama import llm

import pickle 

In [2]:
data_train = load_conll_dataset(split = 'train', cleaned = True)
data_test = load_conll_dataset(split = 'test', cleaned = True)

In [3]:
model = MistralAI()
pt = PT_OutputList(FST_Sentence(training_dataset=data_train, nb_few_shots=3))

sentence = data_test[0]['text']
prompt = pt.get_prompts_runnable(sentence)[0][0]

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA A40, compute capability 8.6
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from llm/models/mistral-7b-v0.1/mistral-7b-v0.1.Q5_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_0     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q5_0     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_0     [  4096, 14336,     1,    

In [8]:
[val for i, val in enumerate(model.model.eval_tokens) if val in values_to_find]

[28755,
 6512,
 9668,
 6512,
 9668,
 6512,
 9668,
 6512,
 9668,
 6512,
 9668,
 6512,
 9668,
 9668,
 9668]

In [20]:
# Afficher n = 0 et n = 2 où LOC est attendu mais on donne du ORG
n = 2
sentence = data_test[n]['text']
print(data_test[n]['text'])
print(data_test[n]['spans'])
get_logits_for_tags(sentence, model, pt)

AL-AIN , United Arab Emirates 1996-12-06
[['AL-AIN', 'LOC'], ['United Arab Emirates', 'LOC']]


([{'PER': 0.9288887977600098,
   'ORG': 4.977176666259766,
   'LOC': 2.1861798763275146,
   'MISC': 5.54980993270874},
  {'PER': -0.43884000182151794,
   'ORG': 2.2182137966156006,
   'LOC': -0.2847512662410736,
   'MISC': 4.719457626342773}],
 {'id': 'cmpl-a2875090-eadd-4a79-a3cf-acfcc18e1a99',
  'object': 'text_completion',
  'created': 1699353777,
  'model': 'llm/models/mistral-7b-v0.1/mistral-7b-v0.1.Q5_0.gguf',
  'choices': [{'text': " [['Al Ain', 'LOC'], ['United Arab Emirates', 'LOC']] ",
    'index': 0,
    'logprobs': None,
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 624,
   'completion_tokens': 26,
   'total_tokens': 650}})

In [77]:
sentence = "Washington is the president of the United States"
sentence_o = " [['Washington', 'LOC'], ['United States', 'LOC']] "
prompt = llama_ner.get_prompts(sentence=sentence, tags = [], n = 0)

output = llm(prompt, stop = ["<end_output>"])

Llama.generate: prefix-match hit

llama_print_timings:        load time =   598.82 ms
llama_print_timings:      sample time =    52.66 ms /    22 runs   (    2.39 ms per token,   417.81 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   598.15 ms /    22 runs   (   27.19 ms per token,    36.78 tokens per second)
llama_print_timings:       total time =   858.95 ms


In [86]:
from ner.evaluating_confidence import get_logits_for_tags
sentences = [
    "Washington is the president of the United States",
    "John is an important person living in Paris"
]
for sent in sentences :
    print(get_logits_for_tags(sent, llm, llama_ner))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   598.82 ms
llama_print_timings:      sample time =    20.90 ms /    22 runs   (    0.95 ms per token,  1052.73 tokens per second)
llama_print_timings: prompt eval time =   134.49 ms /    24 tokens (    5.60 ms per token,   178.46 tokens per second)
llama_print_timings:        eval time =   589.74 ms /    21 runs   (   28.08 ms per token,    35.61 tokens per second)
llama_print_timings:       total time =   983.94 ms


([{'PER': 18.875999450683594, 'ORG': 17.381074905395508, 'LOC': 19.616981506347656, 'MISC': 14.471109390258789}, {'PER': 13.220829010009766, 'ORG': 13.311978340148926, 'LOC': 23.603668212890625, 'MISC': 14.019968032836914}], " [['Washington', 'PER'], ['United States', 'LOC']] ")


Llama.generate: prefix-match hit

llama_print_timings:        load time =   598.82 ms
llama_print_timings:      sample time =    58.87 ms /    20 runs   (    2.94 ms per token,   339.75 tokens per second)
llama_print_timings: prompt eval time =   107.01 ms /    24 tokens (    4.46 ms per token,   224.28 tokens per second)
llama_print_timings:        eval time =   499.01 ms /    19 runs   (   26.26 ms per token,    38.08 tokens per second)
llama_print_timings:       total time =   886.37 ms


([{'PER': 22.131004333496094, 'ORG': 15.82730484008789, 'LOC': 10.609984397888184, 'MISC': 16.390300750732422}, {'PER': 13.270830154418945, 'ORG': 15.87828540802002, 'LOC': 23.22158432006836, 'MISC': 13.704093933105469}], " [['John', 'PER'], ['Paris', 'LOC']] ")


In [48]:
llm.detokenize(list(llm.eval_tokens)[len(llm.tokenize(prompt.encode())):])

b" [['Washington', 'ORG'], ['United States', 'LOC']] <end_output"

In [15]:
model.model.detokenize(list(model.model.eval_tokens)[-23:])

b"  [['JAPAN', 'LOC'], ['CHINA', 'LOC']] <end_output"

In [14]:
list(model.model.eval_tokens)

[1,
 774,
 17121,
 22526,
 714,
 415,
 3638,
 349,
 298,
 9131,
 544,
 272,
 5160,
 936,
 3387,
 297,
 272,
 2296,
 12271,
 28723,
 13,
 27332,
 2223,
 725,
 714,
 3604,
 5541,
 349,
 298,
 9131,
 544,
 272,
 481,
 1218,
 369,
 460,
 2477,
 1338,
 28725,
 6666,
 28725,
 4723,
 442,
 4052,
 2845,
 18834,
 304,
 3825,
 272,
 19810,
 297,
 264,
 1274,
 302,
 8582,
 2815,
 28723,
 560,
 1430,
 21000,
 1658,
 272,
 5160,
 9040,
 304,
 272,
 5846,
 13291,
 378,
 28723,
 13,
 27332,
 8602,
 8048,
 12738,
 714,
 2418,
 368,
 2111,
 528,
 16066,
 2500,
 356,
 272,
 1581,
 1212,
 302,
 19810,
 1550,
 28705,
 13,
 27332,
 2223,
 725,
 714,
 5592,
 28723,
 6441,
 19810,
 460,
 544,
 272,
 2955,
 368,
 541,
 1300,
 297,
 272,
 2245,
 28723,
 1725,
 541,
 347,
 5207,
 24508,
 497,
 28725,
 10578,
 10389,
 28725,
 24138,
 1479,
 6128,
 442,
 776,
 5509,
 2955,
 28723,
 13,
 21830,
 1837,
 19810,
 544,
 272,
 9909,
 368,
 541,
 1300,
 297,
 272,
 2245,
 28723,
 1725,
 541,
 347,
 1955,
 28725,
 14165,